*Step.1 載入相關套件*

In [ ]:
!pip install evaluate
!pip install datasets

In [ ]:
import evaluate
from datasets import Dataset , DatasetDict
from transformers import AutoTokenizer , AutoModelForMultipleChoice , Trainer , TrainingArguments

*Step.2 載入數據*

In [ ]:
c3 = DatasetDict.load_from_disk("./c3")
c3

In [ ]:
c3["train"][0]

In [ ]:
c3.pop("test")

In [ ]:
c3

*Step.3 數據預處理*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

In [ ]:
def process_function(examples):
    # examples, dict, keys: ["context", "quesiton", "choice", "answer"]
    # examples, 1000
    context = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx])
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx)
            question_choice.append(question + " " + choice)
        if len(choices) < 4:
            for _ in range(4 - len(choices)):
                context.append(ctx)
                question_choice.append(question + " " + "不知道")
        labels.append(choices.index(examples["answer"][idx]))
    tokenized_examples = tokenizer(context, question_choice, truncation="only_first", max_length=256, padding="max_length")     # input_ids: 4000 * 256,
    tokenized_examples = {k: [v[i: i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}     # 1000 * 4 *256
    tokenized_examples["labels"] = labels
    return tokenized_examples

In [ ]:
res = c3["train"].select(range(10)).map(process_function, batched=True)
res

In [ ]:
import numpy as np
np.array(res["input_ids"]).shape

In [ ]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

*Step.4 模型建立*

In [ ]:
model = AutoModelForMultipleChoice.from_pretrained("hfl/chinese-macbert-base")

*Step.5 評估函數*

In [ ]:
import numpy as np
accuracy = evaluate.load("accuracy")

def compute_metric(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

*Step.6 訓練參數*

In [ ]:
args = TrainingArguments(
    output_dir = "./multiple_choice",
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    logging_steps =50,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end= True ,
    fp16 = True
)

*Step.7 訓練器*

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_c3["train"],
    eval_dataset = tokenized_c3["validation"],
    compute_metrics = compute_metric

)

*Step.8 模型訓練*

In [ ]:
trainer.train()

*Step.9 模型預測*

In [ ]:
from typing import Any
import torch

class MultiplePieline:

  def __init__(self, model , tokenizer) -> None:
    self.model = model
    self.tokenizer = tokenizer
    self.device = model.device

  def preprocess(self , context , question , choices):
    cs , qcs = [],[]
    for choice in choices:
      cs.append(context)
      qcs.append(question + " " + choice)
    return tokenizer(cs , qcs ,truncation = "only_first" , max_length = 256 , return_tensors="pt")

  def predict(self , inputs):
    inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()}
    return self.model(**inputs).logits

  def postprocess(self , logits , choices):
    predition = torch.argmax(logits, dim=-1).cpu().item()
    return choices[predition]


  def __call__(self, context , question , choices) -> Any:
     inputs = self.preprocess(context , question , choices)
     logits = self.predict(inputs)
     result = self.postprocess(logits , choices)
     return result

In [ ]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [ ]:
pipe("小明在北京上班","小明在哪裡上班?",["北京","上海"])